# Google Gemma 300m & Reranking
Before moving to a reranker, I will test google gemma model, everything stays the same, just changing up the model.

In [1]:
from utils import load_processed_data
docs_for_splitter, questions_ground_truth = load_processed_data("../data/processed/squad_processed.pkl")

from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer
from langchain_text_splitters import RecursiveCharacterTextSplitter

tokenizer = AutoTokenizer.from_pretrained("google/embeddinggemma-300m")
splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer,chunk_size=512, chunk_overlap=75)
splits = splitter.split_documents(docs_for_splitter)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [2]:
import os
from langchain_chroma import Chroma

persist_directory = "./chroma/04_google_gemma"
embed = HuggingFaceEmbeddings(model_name="google/embeddinggemma-300m",
                              encode_kwargs={'normalize_embeddings': True})
if os.path.exists(persist_directory):
    print("Loading existing embeddings...")
    vectorstore = Chroma(
        persist_directory=persist_directory, 
        embedding_function=embed
    )
else:
    print("No existing index found. Generating embeddings...")
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embed,
        persist_directory=persist_directory,
        collection_metadata={"hnsw:space": "cosine"}
    )

retriever = vectorstore.as_retriever()

modules.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

3_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

No existing index found. Generating embeddings...


In [3]:
from utils import evaluate_retrieval

results = evaluate_retrieval(questions_ground_truth,retriever)

Starting evaluation on 2265 questions...


100%|██████████| 2265/2265 [02:56<00:00, 12.82it/s]


--- Evaluation Results ---
MRR: 0.8527
Hit Rate@1: 78.15%
Hit Rate@3: 91.39%
Hit Rate@5: 94.66%
Hit Rate@7: 95.85%
Hit Rate@10: 96.78%


We see great improvement with gemma with significantly shorter time needed to run both embedding and evaluation.<br>
One more thing I will test is reranking. As we see the difference between hit rate@1 and hit rate@10 is about 18%, meaning if we can retrieve 10 paragraphs and rerank them succesfully, we can greatly increase top 1 metric.<br>
I will use cross-encoder for MS Macro as he is quite light with only about 23 million parameters.

In [8]:
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2")

compressor = CrossEncoderReranker(model=model, top_n=10)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, 
    base_retriever=retriever
)

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [11]:
# compression_retriever.invoke(questions_ground_truth[53]['question'])
results_reranker = evaluate_retrieval(questions_ground_truth,compression_retriever)

Starting evaluation on 2265 questions...


100%|██████████| 2265/2265 [12:23<00:00,  3.05it/s]


--- Evaluation Results ---
MRR: 0.9226
Hit Rate@1: 89.05%
Hit Rate@3: 95.32%
Hit Rate@5: 96.42%
Hit Rate@7: 96.64%
Hit Rate@10: 96.78%


Great improvemnt while time needed to run the evaluation is higher then with mixedbread's model.